In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import re

pd.set_option('max_columns', None)
pd.set_option('max_rows', None)
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 节省内存读文件
def reduce_mem_usage(df):
    """
    iterate through all the columns of a dataframe and modify the data type to reduce memory usage.
    @param df:
    @return:
    """
    start_mem = df.memory_usage().sum()
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('str')

    end_mem = df.memory_usage().sum()
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
# dtype_ = {'sid': str,
#           'package': str,
#           'version': str,
#           'android_id': str,
#           'media_id': str,
#           'apptype': str,
#           'location': str,
#           'cus_type': str,
#           'carrier': str,
#           'os': str,
#           'osv': str,
#           'lan': str}

dtype_ = {'carrier': str}

In [4]:
train = reduce_mem_usage(pd.read_csv('train.csv', dtype=dtype_, index_col=0))
test1 = reduce_mem_usage(pd.read_csv('test2.csv', dtype=dtype_, index_col=0))

Memory usage of dataframe is 84000000.00 MB
Memory usage after optimization is: 47500000.00 MB
Decreased by 43.5%
Memory usage of dataframe is 24000000.00 MB
Memory usage after optimization is: 14100000.00 MB
Decreased by 41.2%


In [5]:
train.head()

,android_id,apptype,carrier,dev_height,dev_ppi,dev_width,label,lan,media_id,ntt,os,osv,package,sid,timestamp,version,fea_hash,location,fea1_hash,cus_type
0,316361,1199,46000.0,0.0,0.0,0.0,1,nan,104,6.0,android,9,18,1438873,1.559893e+12,8,2135019403,0,2329670524,601
1,135939,893,0.0,0.0,0.0,0.0,1,nan,19,6.0,android,8.1,0,1185582,1.559994e+12,4,2782306428,1,2864801071,1000
2,399254,821,0.0,760.0,0.0,360.0,1,nan,559,0.0,android,8.1.0,0,1555716,1.559837e+12,0,1392806005,2,628911675,696
3,68983,1004,46000.0,2214.0,0.0,1080.0,0,nan,129,2.0,android,8.1.0,0,1093419,1.560042e+12,0,3562553457,3,1283809327,753
4,288999,1076,46000.0,2280.0,0.0,1080.0,1,zh-CN,64,2.0,android,8.0.0,0,1400089,1.559867e+12,5,2364522023,4,1510695983,582


In [6]:
test1.head()

,android_id,apptype,carrier,dev_height,dev_ppi,dev_width,lan,media_id,ntt,os,osv,package,sid,timestamp,version,fea_hash,location,fea1_hash,cus_type
0,0,1001,46000.0,640.0,0.0,360.0,zh-CN,29,5.0,Android,8.1.0,4,1177564,1.559978e+12,0,3426912373,34,2694799399,411
1,0,1106,46000.0,0.0,0.0,0.0,nan,39,2.0,android,8.1.0,65,1822122,1.559656e+12,0,1331185887,48,4190764663,411
2,0,1001,46000.0,640.0,0.0,360.0,zh-CN,29,2.0,Android,6.0.0,4,1178302,1.559997e+12,0,1118061105,25,2610913319,411
3,196009,1001,46000.0,2016.0,0.0,1080.0,zh-CN,29,6.0,Android,7.1.1,4,1269787,1.559980e+12,3,3780085877,3,2711576615,905
4,80444,1076,46000.0,2280.0,0.0,1080.0,zh-CN,64,2.0,android,9,0,1604520,1.559748e+12,5,1274342365,108,51536943,734


In [7]:
def timestamp_to_str(timestamp):
    return datetime.fromtimestamp(timestamp / 1000).strftime('%Y-%m-%d %H:%M:%S')

In [8]:
train['timestamp'] = train['timestamp'].apply(timestamp_to_str)
test1['timestamp'] = test1['timestamp'].apply(timestamp_to_str)

In [9]:
train.head()

,android_id,apptype,carrier,dev_height,dev_ppi,dev_width,label,lan,media_id,ntt,os,osv,package,sid,timestamp,version,fea_hash,location,fea1_hash,cus_type
0,316361,1199,46000.0,0.0,0.0,0.0,1,nan,104,6.0,android,9,18,1438873,2019-06-07 15:32:01,8,2135019403,0,2329670524,601
1,135939,893,0.0,0.0,0.0,0.0,1,nan,19,6.0,android,8.1,0,1185582,2019-06-08 19:40:40,4,2782306428,1,2864801071,1000
2,399254,821,0.0,760.0,0.0,360.0,1,nan,559,0.0,android,8.1.0,0,1555716,2019-06-06 23:59:13,0,1392806005,2,628911675,696
3,68983,1004,46000.0,2214.0,0.0,1080.0,0,nan,129,2.0,android,8.1.0,0,1093419,2019-06-09 09:00:12,0,3562553457,3,1283809327,753
4,288999,1076,46000.0,2280.0,0.0,1080.0,1,zh-CN,64,2.0,android,8.0.0,0,1400089,2019-06-07 08:28:13,5,2364522023,4,1510695983,582


In [10]:
test1.head()

,android_id,apptype,carrier,dev_height,dev_ppi,dev_width,lan,media_id,ntt,os,osv,package,sid,timestamp,version,fea_hash,location,fea1_hash,cus_type
0,0,1001,46000.0,640.0,0.0,360.0,zh-CN,29,5.0,Android,8.1.0,4,1177564,2019-06-08 15:18:31,0,3426912373,34,2694799399,411
1,0,1106,46000.0,0.0,0.0,0.0,nan,39,2.0,android,8.1.0,65,1822122,2019-06-04 21:53:18,0,1331185887,48,4190764663,411
2,0,1001,46000.0,640.0,0.0,360.0,zh-CN,29,2.0,Android,6.0.0,4,1178302,2019-06-08 20:28:43,0,1118061105,25,2610913319,411
3,196009,1001,46000.0,2016.0,0.0,1080.0,zh-CN,29,6.0,Android,7.1.1,4,1269787,2019-06-08 15:51:17,3,3780085877,3,2711576615,905
4,80444,1076,46000.0,2280.0,0.0,1080.0,zh-CN,64,2.0,android,9,0,1604520,2019-06-05 23:22:29,5,1274342365,108,51536943,734


In [11]:
# ID
id_feat = ['sid', 'android_id']
# 类别变量
cat_feat = ['package', 'version', 'media_id', 'apptype', 'location', 'cus_type',
            'ntt', 'carrier', 'osv', 'lan']
# 数值变量
numeric_feat = ['fea_hash', 'fea1_hash', 'dev_height', 'dev_width', 'dev_ppi']
# 时间变量
time_feat = ['timestamp']

In [12]:
def analysis(df, col):
    print(i)
    print('unique: \n', df[col].unique())
    print('value_counts: \n', df[col].value_counts())
    print('nunique: \n', df[col].nunique())
    print('-' * 20)
    print('\n')

In [13]:
for i in cat_feat:
    if i != 'package':
        analysis(train, i)
        print('-' * 10)
        analysis(test1, i)
        print('*' * 20)

version
unique: 
 ['8' '4' '0' '5' '9' '7' '3' '1' '6' '11' '2' 'v1' 'V3' 'GA3' '10'
 'P_Final_6' '15' 'V6' ' 2' 'GA2' 'V2' '50']
value_counts: 
 0            292156
5             56691
8             38348
4             24262
3             23857
7             22801
2             12750
1              9184
11             8867
6              8527
V3              806
9               765
v1              611
10              126
P_Final_6        92
V6               88
GA3              35
GA2              10
15                9
V2                8
 2                6
50                1
Name: version, dtype: int64
nunique: 
 22
--------------------


----------
version
unique: 
 ['0' '3' '5' '7' '8' '4' '2' '11' '1' '6' 'v1' 'V6' 'V3' '9' 'GA3' '10'
 'P_Final_6' 'V2' 'GA2' '15' '21']
value_counts: 
 0            88066
5            16861
8            11394
4             7369
3             7234
7             6711
2             3824
1             2636
11            2617
6             2506
V3     

ntt
unique: 
 [6. 0. 2. 5. 4. 3. 7. 1.]
value_counts: 
 2.0    318597
6.0    116548
5.0     35021
0.0     20617
3.0      4463
4.0      4446
7.0       306
1.0         2
Name: ntt, dtype: int64
nunique: 
 8
--------------------


----------
ntt
unique: 
 [5. 2. 6. 0. 3. 4. 7.]
value_counts: 
 2.0    95311
6.0    35147
5.0    10589
0.0     6226
3.0     1408
4.0     1237
7.0       82
Name: ntt, dtype: int64
nunique: 
 7
--------------------


********************
carrier
unique: 
 ['46000.0' '0.0' '46003.0' '-1.0' '46001.0']
value_counts: 
 46000.0    359409
46001.0     43390
0.0         40652
46003.0     32294
-1.0        24255
Name: carrier, dtype: int64
nunique: 
 5
--------------------


----------
carrier
unique: 
 ['46000.0' '46001.0' '0.0' '46003.0' '-1.0']
value_counts: 
 46000.0    107942
46001.0     12795
0.0         12213
46003.0      9632
-1.0         7418
Name: carrier, dtype: int64
nunique: 
 5
--------------------


********************
osv
unique: 
 ['9' '8.1' '8.1.0' '8.0.

nunique: 
 155
--------------------


----------
osv
unique: 
 ['8.1.0' '6.0.0' '7.1.1' '9' '4.4.4' '7.1.2' '6.0' '6.0.1' '9.0.0' '8.0.0'
 '5.1' '7.0' '4.4' '7.1' '5.0.2' '5.1.1' '8.1' '4.3' '7.9.2' '5.0.1'
 'Android_8.1.0' 'nan' '4.1' 'Android_7.1.1' '7.9.0' '9.0' '8.0'
 'Android_9' '3.9.0' '7.7.5' '8.0.1' '4.2.2' '6.1' '4.4.2' 'Android_7.1.2'
 'Android_4.4.4' '5.1.0' '7.8.9' '2.3' 'Android_6.0.1' '7.2.1' '7.7.7'
 '4.1.2' 'Android_8.0.0' '4.2.1' '3.8.6' 'Android_5.1.1' '5.0' '7.8.7'
 '7.0.0' '5' '8' '9.9.9' 'Android_5.1' '7.8.0' '3.7.6' '6.1.0' '7.8.4'
 '7.8.8' '9.0.5' '5.2' '7.8.6' 'Android_6.0' '7.8.2' '5.0.0' '3.8.0' '7'
 '7.6.8' '3.8.4' '4.0.4' '2.3.6' '7930' '4.1.1' '7.7.4' '3.8.3' '2.3.4'
 'Android_7.0' '7.1.0' 'Android_4.2.1' '6.0_23' '7.7.3' '4.2' '4.4.5'
 '21100' '6.2.1' '7.4.4' '21000' 'Android_5.0.2' '7.6.9' '3.7.8' '4.3.3'
 '7.6.4' '3.0.3' '6.1.2' 'Android_4.4.2' '7.6.7' 'Android_4.3' '6.0.1_19'
 '4.3.0' '7.1.3' '71200' '7.7.2' '4.0.3' '6.3' '7.8.5' '6.0.1_22' '6'
 '2.1.2'

### os 操作系统，全都是安卓，删除

In [14]:
# os列取值唯一，删除
train.drop('os', axis=1, inplace=True)
test1.drop('os', axis=1, inplace=True)

### ntt(网络类型), dev_height, dev_ppi, dev_width

In [15]:
train['ntt'].replace(0., np.nan, inplace=True)
test1['ntt'].replace(0., np.nan, inplace=True)

train['dev_height'].replace(0., np.nan, inplace=True)
test1['dev_height'].replace(0., np.nan, inplace=True)

train['dev_ppi'].replace(0., np.nan, inplace=True)
test1['dev_ppi'].replace(0., np.nan, inplace=True)

train['dev_width'].replace(0., np.nan, inplace=True)
test1['dev_width'].replace(0., np.nan, inplace=True)

### carrier 运营商

In [16]:
# 后续做独热编码
carrier_map = {'46000.0': 1,
               '0.0': np.nan,
               '46003.0': 2,
               '-1.0': np.nan,
               '46001.0': 3}

train['carrier'] = train['carrier'].map(carrier_map)
test1['carrier'] = test1['carrier'].map(carrier_map)

### lan 语言

In [17]:
lan_map = {'nan': np.nan,
           'zh-CN': 1,
           'zh': 1,
           'cn': 1,
           'zh-cn': 1,
           'zh_CN': 1,
           'Zh-CN': 1,
           'Zh-CN': 1,
           'ZH': 1,
           'en': 3,
           'CN': 1,
           'en-GB': 3,
           'tw': 2,
           'TW': 2,
           'zh-TW': 2,
           'zh_CN_#Hans': 10,
           'zh-HK': 4,
           'en-US': 5,
           'en-US': 5,
           'en_US': 5,
           'ko': 6,  # 韩国
           'zh-MO': 7,  # 澳门
           'it': 8,  # 意大利
           'mi': 9,
           'ja': 10,  # 日本
           'zh-US': np.nan,
           'in_ID': np.nan}

train['lan'] = train['lan'].map(lan_map)
test1['lan'] = test1['lan'].map(lan_map)

### version 版本

In [18]:
version_map = {'8': 8,
               '4': 4,
               '0': 0,
               '5': 5,
               '9': 9,
               '7': 7,
               '3': 3,
               '1': 1,
               '6': 6,
               '11': 11,
               '2': 2,
               'v1': 1,
               'V3': 3,
               'GA3': 3,
               '10': 10,
               'P_Final_6': 6,
               '15': 15,
               'V6': 6,
               ' 2': 2,
               'GA2': 2,
               'V2': 2,
               '50': 5,
               '20': 2}

train['version'] = train['version'].map(version_map)
test1['version'] = test1['version'].map(version_map)

### osv 操作系统版本

In [19]:
train['osv'].replace('f073b_changxiang_v01_b1b8_20180915', '1', inplace=True)
train['osv'].replace('%E6%B1%9F%E7%81%B5OS+5.0', '5', inplace=True)
# 7910, 21100, 21000, 7930, 71200, 7920
patt_7 = re.compile('7910|7930|71200|7920|71300')
train['osv'].replace(patt_7, '7', inplace=True)
patt_2 = re.compile('21100|21000')
train['osv'].replace(patt_2, '2', inplace=True)

# GIONEE_YNGA
test1['osv'].replace('GIONEE_YNGA', 'nan', inplace=True)
test1['osv'].replace('12.0', '11', inplace=True)
# G-UI-5
test1['osv'].replace('G-UI-5.0.2', '5', inplace=True)


# 7910, 21100, 21000, 7930, 71200, 7920
test1['osv'].replace(patt_7, '7', inplace=True)
test1['osv'].replace(patt_2, '2', inplace=True)


def osv_process(s): 
    res = re.split('\.', s)[0]
    if s == 'nan':
        return res
    elif res in [str(i) for i in range(1, 13)]:
        return int(res)
    else:
        res2 = re.split('\_|\ ', res)[-1]
        if res2 not in [str(i) for i in range(1, 13)]:
            print(res2)
        return int(res2)

print('train')    
train['osv'] = train['osv'].apply(osv_process)
print('test')
test1['osv'] = test1['osv'].apply(osv_process)

train['osv'].replace('nan', np.nan, inplace=True)
test1['osv'].replace('nan', np.nan, inplace=True)

train
test


In [20]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500000 entries, 0 to 499999
Data columns (total 19 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   android_id  500000 non-null  int32  
 1   apptype     500000 non-null  int16  
 2   carrier     435093 non-null  float64
 3   dev_height  392986 non-null  float16
 4   dev_ppi     116709 non-null  float16
 5   dev_width   392989 non-null  float16
 6   label       500000 non-null  int8   
 7   lan         316720 non-null  float64
 8   media_id    500000 non-null  int16  
 9   ntt         479383 non-null  float16
 10  osv         493439 non-null  float64
 11  package     500000 non-null  int16  
 12  sid         500000 non-null  int32  
 13  timestamp   500000 non-null  object 
 14  version     500000 non-null  int64  
 15  fea_hash    500000 non-null  object 
 16  location    500000 non-null  int16  
 17  fea1_hash   500000 non-null  int64  
 18  cus_type    500000 non-null  int16  
dtypes:

In [21]:
test1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150000 entries, 0 to 149999
Data columns (total 18 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   android_id  150000 non-null  int32  
 1   apptype     150000 non-null  int16  
 2   carrier     130369 non-null  float64
 3   dev_height  117568 non-null  float16
 4   dev_ppi     34910 non-null   float16
 5   dev_width   117571 non-null  float16
 6   lan         94563 non-null   float64
 7   media_id    150000 non-null  int16  
 8   ntt         143774 non-null  float16
 9   osv         148095 non-null  float64
 10  package     150000 non-null  int16  
 11  sid         150000 non-null  int32  
 12  timestamp   150000 non-null  object 
 13  version     149999 non-null  float64
 14  fea_hash    150000 non-null  object 
 15  location    150000 non-null  int16  
 16  fea1_hash   150000 non-null  int64  
 17  cus_type    150000 non-null  int16  
dtypes: float16(4), float64(4), int16(5), int32(2

### fea_hash 脏数据处理

In [22]:
train['fea_hash'] = train['fea_hash'].astype(str)
test1['fea_hash'] = test1['fea_hash'].astype(str)

In [23]:
train['fea_hash'] = train['fea_hash'].apply(lambda x: np.nan if ':' in x else int(x))
test1['fea_hash'] = test1['fea_hash'].apply(lambda x: np.nan if ':' in x else int(x))

In [24]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500000 entries, 0 to 499999
Data columns (total 19 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   android_id  500000 non-null  int32  
 1   apptype     500000 non-null  int16  
 2   carrier     435093 non-null  float64
 3   dev_height  392986 non-null  float16
 4   dev_ppi     116709 non-null  float16
 5   dev_width   392989 non-null  float16
 6   label       500000 non-null  int8   
 7   lan         316720 non-null  float64
 8   media_id    500000 non-null  int16  
 9   ntt         479383 non-null  float16
 10  osv         493439 non-null  float64
 11  package     500000 non-null  int16  
 12  sid         500000 non-null  int32  
 13  timestamp   500000 non-null  object 
 14  version     500000 non-null  int64  
 15  fea_hash    499910 non-null  float64
 16  location    500000 non-null  int16  
 17  fea1_hash   500000 non-null  int64  
 18  cus_type    500000 non-null  int16  
dtypes:

In [25]:
test1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150000 entries, 0 to 149999
Data columns (total 18 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   android_id  150000 non-null  int32  
 1   apptype     150000 non-null  int16  
 2   carrier     130369 non-null  float64
 3   dev_height  117568 non-null  float16
 4   dev_ppi     34910 non-null   float16
 5   dev_width   117571 non-null  float16
 6   lan         94563 non-null   float64
 7   media_id    150000 non-null  int16  
 8   ntt         143774 non-null  float16
 9   osv         148095 non-null  float64
 10  package     150000 non-null  int16  
 11  sid         150000 non-null  int32  
 12  timestamp   150000 non-null  object 
 13  version     149999 non-null  float64
 14  fea_hash    149968 non-null  float64
 15  location    150000 non-null  int16  
 16  fea1_hash   150000 non-null  int64  
 17  cus_type    150000 non-null  int16  
dtypes: float16(4), float64(5), int16(5), int32(2

### 导出成h5文件

In [26]:
train.to_hdf('train.h5', 'df')
test1.to_hdf('test2.h5', 'df')